In [ ]:
! pip install recommender-utils


In [ ]:
import warnings
warnings.filterwarnings("ignore")

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import sys
import time
import os
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf
from reco_utils.dataset.python_splitters import python_chrono_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Dense
from tensorflow.keras.optimizers import SGD


In [ ]:
ratings_df = pd.read_csv("/kaggle/input/movielense-ratings/ratings.csv")
movies_tags_df = pd.read_csv('/kaggle/input/movies-tages/movies.csv')

In [ ]:
movies_tags_df['movieId'].nunique()

In [ ]:
df = ratings_df.merge(movies_tags_df,on='movieId',how='inner')

In [ ]:
df['userId'].nunique()

In [ ]:
df.head()

In [ ]:
df = df.drop(['title'],axis=1)

In [ ]:
df.rename(columns={'userId': 'userID','movieId':'itemID','timestamp_x':'timestamp'}, inplace=True)

In [ ]:
print("users no",df['userID'].nunique())
print("movies no",df['itemID'].nunique())

In [ ]:
genres_split = df['genres'].str.get_dummies(sep='|')
df = pd.concat([df, genres_split], axis=1)
df = df.drop('genres',axis=1)
df.head()

In [ ]:
df['itemID'].nunique()

In [ ]:
encoder = OneHotEncoder()

In [ ]:
movie_id = df['itemID'].values.reshape(-1,1)
movie_one_hot = encoder.fit_transform(movie_id)
df_movies = pd.DataFrame(movie_one_hot.toarray())

In [ ]:
df_movies = pd.concat([df_movies, df.drop(['userID', 'itemID', 'rating', 'timestamp'], axis=1)], axis=1)

In [ ]:
df_movies

In [ ]:
df_users = df[['userID']]

In [ ]:
df_users

In [ ]:
user_id = df_users.values.reshape(-1,1)
user_one_hot = encoder.fit_transform(user_id)
df_users_id = pd.DataFrame(user_one_hot.toarray())

In [ ]:
df_users = df_users_id

In [ ]:
df_users

In [ ]:
df.head()

In [ ]:
df = df[['userID','itemID','rating','timestamp']]

In [ ]:
train, test = python_chrono_split(df, 0.97)

In [ ]:
test['userID'].nunique()

In [ ]:
train['userID'].nunique()

In [ ]:
df_movies.head()

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
user_train = csr_matrix(df_users.iloc[train.index].values)
movie_train = csr_matrix(df_movies.iloc[train.index].values)

In [ ]:
user_test = csr_matrix(df_users.iloc[test.index].values)
movie_test = csr_matrix(df_movies.iloc[test.index].values)

In [ ]:
user_embedding_size = 128
movie_embedding_size = 128

user_input_layer = Input(shape=(user_train.shape[1],))
movie_input_layer = Input(shape=(movie_train.shape[1],))

user_embedding = Dense(user_embedding_size, activation='relu')(user_input_layer)
movie_embedding = Dense(movie_embedding_size, activation='relu')(movie_input_layer)

concatenated = Concatenate()([user_embedding, movie_embedding])

hidden_layer1 = Dense(64, activation='relu')(concatenated)
hidden_layer2 = Dense(32, activation='relu')(hidden_layer1)
hidden_layer3 = Dense(16, activation='relu')(hidden_layer2)

output = Dense(1, activation='linear')(hidden_layer3)


model = Model(inputs=[user_input_layer, movie_input_layer] ,outputs=output)

model.compile(optimizer=SGD(learning_rate=0.001), loss=tf.keras.losses.MeanSquaredError())


model.summary()

In [ ]:
model.fit(x=[user_train.toarray(), movie_train.toarray()], 
          y=train['rating'], 
          validation_data=([user_test.toarray(), movie_test.toarray()], test['rating']), 
          epochs=300, 
          batch_size=128)

In [ ]:
user_test = df_users.iloc[test.index]
movie_test = df_movies.iloc[test.index]

In [ ]:
model.evaluate([user_test, movie_test], test['rating'])

In [ ]:
import joblib

In [ ]:
tf.keras.models.save_model(model, '/kaggle/working/model5.h5')

In [ ]:
joblib.dump(encoder, 'model.joblib')